In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split

C:\Users\PC\OneDrive\Desktop\NLP-CODE\myvenv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Sample Text Data (IMDB or Custom)

In [2]:
texts = [
    "I love this movie",
    "This movie is boring",
    "Amazing story and characters",
    "Waste of time"
]

labels = [1, 0, 1, 0]  # 1=positive, 0=negative


Tokenize Text

In [3]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences so all inputs have same length
X = pad_sequences(sequences, maxlen=5)
y = np.array(labels)


Train-Test Split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


Build LSTM Model

In [5]:
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=64, input_length=5))  # embedding layer
model.add(LSTM(50, return_sequences=False))  # LSTM layer
model.add(Dense(1, activation='sigmoid'))  # output


C:\Users\PC\OneDrive\Desktop\NLP-CODE\myvenv\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Embedding	Converts words → vectors
LSTM 50	Processes sequences, remembers context
Dense 1	Outputs probability of positive review

Compile Model

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Train Model

In [7]:
history = model.fit(X_train, y_train, epochs=10, batch_size=2, validation_split=0.1)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 1.0000 - loss: 0.6912 - val_accuracy: 1.0000 - val_loss: 0.6889
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 1.0000 - loss: 0.6870 - val_accuracy: 1.0000 - val_loss: 0.6892
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 1.0000 - loss: 0.6828 - val_accuracy: 1.0000 - val_loss: 0.6895
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step - accuracy: 1.0000 - loss: 0.6785 - val_accuracy: 1.0000 - val_loss: 0.6897
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 1.0000 - loss: 0.6741 - val_accuracy: 1.0000 - val_loss: 0.6899
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step - accuracy: 1.0000 - loss: 0.6695 - val_accuracy: 1.0000 - val_loss: 0.6899
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step - accuracy: 1.0000 - loss: 0.6646 - val_accuracy: 1.0000 - val_loss: 0.6899
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - accuracy: 1.0000 - loss: 0.6595 - val_accuracy: 1.0000 - val_loss: 0.

Evaluate Model

In [8]:
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.0000e+00 - loss: 0.7004
Test Accuracy: 0.0


Song Lyrics Generator Using LSTM

Prepare Text

In [9]:
text = """I love the way you move
I love the way you groove
You make me feel alive
Let's dance until sunrise"""


Tokenize & Create Sequences

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


Pad Sequences

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_seq_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')

# Split predictors and label
X = input_sequences[:, :-1]
y = input_sequences[:, -1]


Convert y to One-Hot Encoding

In [12]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=total_words)


Build LSTM Model

In [13]:
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_seq_len-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')


Train Model

In [14]:
model.fit(X, y, epochs=500, verbose=1)


Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 2.7688
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 2.7630
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 2.7572
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 2.7511
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - loss: 2.7449
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - loss: 2.7382
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step - loss: 2.7312
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - loss: 2.7236
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - loss: 2.7154
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 2.7065
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - loss: 2.6966
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - loss: 2.6858
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - loss: 2.6739
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - loss: 2.6606
Epoch 15/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - loss: 2.6458
Epoch 1

Generate New Lyrics

Generate New Lyrics

In [15]:
def generate_lyrics(seed_text, next_words, model, max_seq_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)[0]
        output_word = tokenizer.index_word[predicted]
        seed_text += " " + output_word
    return seed_text


In [16]:
print(generate_lyrics("I love", 5, model, max_seq_len))


I love the way you move groove
